In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib.pyplot as plt
%matplotlib inline
#from collections import Counter
#import itertools
#import ast
#import warnings

#import xgboost as xgb
#import catboost as cat
#import lightgbm as lgb

#from scipy.stats import norm, skew
#from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold, cross_validate, cross_val_score
#from sklearn.decomposition import PCA#, TruncatedSVD
#from sklearn.preprocessing import Normalizer, LabelEncoder, OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler, Imputer

#from sklearn.pipeline import make_pipeline, Pipeline
#from sklearn.metrics import mean_squared_error
#from sklearn.ensemble import RandomForestRegressor, ExtraTreesClassifier
#from sklearn.datasets import make_classification

#from sklearn.neighbors import KNeighborsRegressor
#from functools import reduce
#from sklearn import (datasets,discriminant_analysis, neighbors)
#from sklearn.linear_model import LinearRegression#, ElasticNetCV

import json
#import operator
#import time
#import os

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

seed = 42

In [2]:
#load data

towns = pd.read_json(r'data\towns\towns.json', encoding = 'utf-8')
category = pd.read_json (r'data\guidle\category.json', encoding = 'utf-8')
event = pd.read_json (r'data\guidle\event.json', encoding = 'utf-8')
event_cat = pd.read_json (r'data\guidle\event_category.json', encoding = 'utf-8')

#merge to one full dataframe
event = pd.merge(event, event_cat, how='left', on='event_id')
df_full = pd.merge(event, category, how='left', on='category_id')

#drop some columns I won't be analysing at the moment / make small version

#df = df_full[['event_id','title_en_x','address_city','date', 'start_time', 'end_time', 'price_information', 'short_description_en','long_description_en','title_en_y','parent_category_id']].copy()
df = df_full.copy()
df.fillna('',inplace=True)

#df = df_raw.copy()
#test = test_raw.copy()
#both = pd.concat([df, test], axis=0)


In [3]:
#look at the data

df.info()
df.head(50)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49488 entries, 0 to 49487
Data columns (total 33 columns):
address_address_line    49488 non-null object
address_city            49488 non-null object
address_country         49488 non-null object
address_latitude        49488 non-null object
address_longitude       49488 non-null object
address_street          49488 non-null object
address_venue_name      49488 non-null object
address_zip             49488 non-null int64
date                    49488 non-null datetime64[ns]
end_time                49488 non-null object
event_id                49488 non-null int64
homepage                49488 non-null object
long_description_de     49488 non-null object
long_description_en     49488 non-null object
long_description_fr     49488 non-null object
long_description_it     49488 non-null object
price_information       49488 non-null object
short_description_de    49488 non-null object
short_description_en    49488 non-null object
short_descr

,address_address_line,address_city,address_country,address_latitude,address_longitude,address_street,address_venue_name,address_zip,date,end_time,event_id,homepage,long_description_de,long_description_en,long_description_fr,long_description_it,price_information,short_description_de,short_description_en,short_description_fr,short_description_it,start_time,thumbnail_url,title_de_x,title_en_x,title_fr_x,title_it_x,category_id,parent_category_id,title_de_y,title_en_y,title_fr_y,title_it_y
0,Micheletto Sandro,Illgau,Schweiz,46.9873,8.72773,Bacheggli,Stoos-Muotatal Tourismus GmbH,6434,2019-09-27,,2014423,https://www.s-a-m.ch/Termine/EventsDetails/EID...,"Wie angekündigt, hat sich die SAM-Sportkommiss...","As announced, following the cancellation of th...","Wie angekündigt, hat sich die SAM-Sportkommiss...","Wie angekündigt, hat sich die SAM-Sportkommiss...",Saturday:\r\nAdults CHF 15.00\r\nChildren up t...,Die noch ausstehende Bewilligung des Kantons i...,The still outstanding permit of the canton has...,Die noch ausstehende Bewilligung des Kantons i...,Die noch ausstehende Bewilligung des Kantons i...,2019-09-29 19:00:00,https://d37dhr5745n0y2.cloudfront.net/c/00/b1/...,Supermoto Ried Muotathal SZ,Supermoto Ried Muotathal SZ,Supermoto Ried Muotathal SZ,Supermoto Ried Muotathal SZ,,,,,,
1,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,879,,Museum,Museums & Attractions,Musées & attractions,Musei & Monumenti
2,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,903,879,Museumsführung,Museum tour,Visite muséal,Visita guidata del Museo
3,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,Where does the Swiss cross come from? Why do p...,2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,Swissness. Tradition and innovation,2171,1147,"Wissen, Informatik & Umwelt","Knowledge, computer science & environment","Connaissances, Informatique & environnement","Conoscenza, Informatica & ambiente"
4,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,

In [4]:
df.groupby('title_en_y').count().sort_values('event_id', ascending=False)

#exhibitions, Arts, Sightseeing & city tour, Sport, History, Stage, Museums & attractions, 
#Knowledge, computer science & environment

recreation_types = ['hiking', 'skiing', 'biking', 'watersports', 'wellness', 'shopping', 'panorama_trips', 'city_trips', 'nature_and_parks', 'culture']
# -> cities will have these as dummy categories

event_types = ['exhibitions', 'sightseeing', 'sport', 'museums', 'science', 'gastronomy', 'concerts', 'fair_and_market']
# -> events will have these as dummy categories


# -> keywords feature with list of searchable strings

#   two dfs: merged event-df and cities-recreation-info-df, these will be compared with traveltime api

#user can search for a either a regular or a railaway surprise_me destination. 
#in either case: mark the event/recreation options they're interested in, give time and price, 
#also searchable by travle distance, in which case will check with travel distance api using the towns df
#speech-to-text option. the strings sentences are first checked against the recreation_types, then the event_types and its keywords
#could get city recreation keywords to associate with each category




In [5]:
#re-categorize events

event_cat = {
    'exhibitions': ['Exhibitions','Arts', 'Other exhibitions', 'Art & design', 'Permanent exhibition', 'Special exhibition'] , 
    'sightseeing': ['Sightseeing & city tour', 'Excursion'], 
    'sport': ['Sports'], 
    'museums': ['History', 'Museums & Attractions', 'Other museum & attraction', 'Permanent exhibition', 'Special exhibition' ], 
    'science': ['Knowledge, computer science & environment', 'Nature / Environment', 'Cooking, Food & Taste'], 
    'gastronomy': ['Culinary art', 'Special food offers', 'More Food Specials'], 
    'concerts' : ['Concerts others','Other music ads', 'Stage'] , 
    'fair_and_market': ['Fair & market', 'Crafts / Gold / jewelry / fashion', 'Society', 'Man / society']
}

text_cols = ['long_description_de', 'long_description_en',
       'long_description_fr', 'long_description_it', 'price_information',
       'short_description_de', 'short_description_en', 'short_description_fr',
       'short_description_it','title_de_x',
       'title_en_x', 'title_fr_x', 'title_it_x','title_de_y', 'title_en_y', 'title_fr_y',
       'title_it_y']
    
def get_cats(d):
    
    #Change overview into list of lower case words, get rid of non-alphanumeric characters. 
    #i=short_description_en
    #d.loc[d[i].notnull(),i]=d.loc[d[i].notnull(),i].apply(lambda x : [''.join(ch for ch in y if ch.isalnum()).lower() for y in str(x).split(' ')])
    #d['len_overview']= d[i].apply(lambda x:len(x) if x!=0 else 0)
    
    
    for i in event_types:
        d[i] = d.title_en_y.apply(lambda x: x in event_cat[i])
        
     
    for col in text_cols:
        d.loc[d[col].notnull(),col]=d.loc[d[i].notnull(),col].apply(lambda x : [''.join(ch for ch in y if ch.isalnum()).lower() for y in str(x).split(' ')])
    
    
           
    
    #d['overview'] = d.title_en_y.apply(lambda x: [''.join(ch for ch in y if ch.isalnum()).lower() for y in str(x).split(' ')])
    
    #rename some stuff, drop unnecessary categories
    #d.rename(columns = {'':'','':'','':''},inplace = True)
    
    #d.index = d['']
    
    #d = d.drop(['','',''],axis=1)
    
    return d


df = get_cats(df)


In [6]:
df.to_json(path_or_buf='event_data.json')#, encoding = 'utf-8')

,address_address_line,address_city,address_country,address_latitude,address_longitude,address_street,address_venue_name,address_zip,date,end_time,event_id,homepage,long_description_de,long_description_en,long_description_fr,long_description_it,price_information,short_description_de,short_description_en,short_description_fr,short_description_it,start_time,thumbnail_url,title_de_x,title_en_x,title_fr_x,title_it_x,category_id,parent_category_id,title_de_y,title_en_y,title_fr_y,title_it_y,exhibitions,sightseeing,sport,museums,science,gastronomy,concerts,fair_and_market
0,Micheletto Sandro,Illgau,Schweiz,46.9873,8.72773,Bacheggli,Stoos-Muotatal Tourismus GmbH,6434,2019-09-27,,2014423,https://www.s-a-m.ch/Termine/EventsDetails/EID...,"[wie, angekündigt, hat, sich, die, samsportkom...","[as, announced, following, the, cancellation, ...","[wie, angekündigt, hat, sich, die, samsportkom...","[wie, angekündigt, hat, sich, die, samsportkom...","[saturdayadults, chf, 1500children, up, to, 14...","[die, noch, ausstehende, bewilligung, des, kan...","[the, still, outstanding, permit, of, the, can...","[die, noch, ausstehende, bewilligung, des, kan...","[die, noch, ausstehende, bewilligung, des, kan...",2019-09-29 19:00:00,https://d37dhr5745n0y2.cloudfront.net/c/00/b1/...,"[, supermoto, ried, muotathal, sz]","[, supermoto, ried, muotathal, sz]","[, supermoto, ried, muotathal, sz]","[, supermoto, ried, muotathal, sz]",,,[],[],[],[],False,False,False,False,False,False,False,False
1,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",[],"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,"[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]",879,,[museum],"[museums, , attractions]","[musées, , attractions]","[musei, , monumenti]",False,False,False,True,False,False,False,False
2,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",[],"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,"[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]","[swissness, tradition, and, innovation]",903,879,[museumsführung],"[museum, tour]","[visite, muséal]","[visita, guidata, del, museo]",False,False,False,False,False,False,False,False
3,Landesmuseum Zürich,Zürich,Schweiz,47.379,8.541,Museumstrasse 2,Schweizerisches Nationalmuseum,8001,2019-09-27,2019-09-29 12:00:00,2014567,,"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",[],"[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...","[where, does, the, swiss, cross, come, from, w...",2019-09-29 11:00:00,https://d37dhr5745n0y2.cloudfront.net/0/88/ec/...,"[swiss

In [ ]:
#ignore

In [7]:
keywords_dict = {}

for cat in event_types:
    
    d = df[df[cat]==True][text_cols].copy()
    
    words = []
    
    for col in text_cols:
        words.append(list(set([a for b in df[col].tolist() for a in b])))
    

In [8]:
#words_most_used = Counter(words).most_common(100)
    
#keyword_list = []
#for i in words_most_used:
    #keyword_list.append(i[0])
        
#keywords_dict[cat] = keyword_list



In [9]:
#debug

keywords_dict = {}

cat = event_types[0]
    
d = df[df[cat]==True][text_cols].copy()
    
words = []
    
for col in text_cols:
    words.append(list(set([a for b in df[col].tolist() for a in b])))
    
    
#words_most_used = Counter(words).most_common(100)
    
#keyword_list = []
#for i in words_most_used:
#    keyword_list.append(i[0])

#keywords_dict[cat] = keyword_list


#print (type(df.val.ix[0]))
#<class 'list'>

#print (df.val.tolist())
#[['val1', 'val2'], ['val33', 'val9', 'val6'], ['val2', 'val6', 'val7']]

#print (list(set([a for b in df.val.tolist() for a in b])))
#['val7', 'val1', 'val6', 'val33', 'val2', 'val9']

In [11]:

#df_new[df_new['concerts']==True]

NameError: name 'df_new' is not defined

In [ ]:
#def get_word_usage(d):

#title_oft=Counter(" ".join(df["title_en_x"].str.lower()).split()).most_common(100)
    
title_words = " ".join(df["title_en_x"].str.lower()).split() + " ".join(df["short_description_en"].str.lower()).split()
title_keywords= [x for x in title_words if len(x) >= 5]

title_oft = Counter(title_keywords).most_common(100)

#title_oft[0][0]


In [ ]:
#def get_word_usage(d):

#title_oft=Counter(" ".join(df["title_en_x"].str.lower()).split()).most_common(100)
    
title_words = " ".join(df["long_description_en"].str.lower()).split()
title_keywords= [x for x in title_words if len(x) >= 5]

title_oft = Counter(title_keywords).most_common(100)

In [ ]:
#def get_word_usage(d):

#title_oft=Counter(" ".join(df["title_en_x"].str.lower()).split()).most_common(100)


descr_s_words = " ".join(df["short_description_en"].str.lower()).split()
descr_s_keywords= [x for x in title_words if len(x) >= 5]

descr_s_oft = Counter(title_keywords).most_common(100)

descr_s_oft

In [ ]:
descr_l_words = " ".join(df["long_description_en"].str.lower()).split()
descr_l_keywords= [x for x in title_words if len(x) >= 5]

descr_l_oft = Counter(title_keywords).most_common(100)

descr_l_oft

In [ ]:
df.groupby('address_city').count().sort_values('event_id', ascending=False)

In [ ]:


nums = ['num_languages','num_genres','num_keywords','num_companies','num_countries','num_cast']

df=get_mains(df)
test=get_mains(test)

df_d =df.copy()
test_d=test.copy()
both_d = pd.concat([df.copy(), test.copy()], axis=0).drop(['rev','top_rev'],axis=1)

#df.info()
#test.info()
both.info()
df.head()

In [ ]:
#category.isna().sum().sort_values() #drop from category: title_de, title_fr, title_it
event_short.isna().sum().sort_values()

In [ ]:
keywords_dict = {
    'exhibitions': [] , 
    'sightseeing': [], 
    'sport': [], 
    'museums': [], 
    'science': [], 
    'gastronomy': [], 
    'concerts' : [] , 
    'fair_and_market': []
}

In [ ]:
for cat in event_types:
    
    d = df[df[cat]==True][['long_description_de', 'long_description_en',
       'long_description_fr', 'long_description_it', 'price_information',
       'short_description_de', 'short_description_en', 'short_description_fr',
       'short_description_it','title_de_x',
       'title_en_x', 'title_fr_x', 'title_it_x','title_de_y', 'title_en_y', 'title_fr_y',
       'title_it_y']].copy()
    
    words = []
    
    all_words = " ".join(df["title_en_x"].str.lower()).split() + " ".join(df["short_description_en"].str.lower()).split()
    all_keywords= [x for x in all_words if len(x) >= 5]

    keywords = Counter(title_keywords).most_common(1000)

    keyword_list = []
    for i in keywords:
        keyword_list.append(i[0])
    
    for  col in d.columns:
        words.
        
        d.loc[d[col].notnull(),col]=d.loc[d[i].notnull(),col].apply(lambda x : [''.join(ch for ch in y if ch.isalnum()).lower() for y in str(x).split(' ')])
    
    
    for word in keyword_list:
        if word in df.loc[df['long_description_en'].notnull(), 'long_description_en'] and df[cat,i] == True:
            
        
       

In [ ]:
df.columns


In [ ]:
railaway_types =['Rail & boat',
 'Games & fun',
 'Mountains & views',
 'Adventure & panorama trips',
 'Nature & sights of interest',
 'Zoos & animal parks',
 'Bike & e-bike',
 'Wellness & relaxation',
 'Hiking',
 'Art, culture & museums',
 'Trade fairs & folk festivals',
 'Concerts, musicals, festivals',
 'Sports events',
 'Markets, shopping & culinary art',
 'Short trips in Switzerland',
 'City trips',
 'Family excursions',
 'School excursions',
 'Group excursions']

In [ ]:
all_words = " ".join(df["title_en_x"].str.lower()).split() + " ".join(df["short_description_en"].str.lower()).split()
all_keywords= [x for x in all_words if len(x) >= 5]

keywords = Counter(title_keywords).most_common(1000)

keyword_list = []
for i in keywords:
    keyword_list.append(i[0])